# K-means: Intra-cluster Minimization

We consider a set of points

$$
x_1, \dots, x_n \in \mathbb{R}^d
$$

which are partitioned into $k$ clusters $C_1, \dots, C_k$.


## Optimization Problem

The K-means objective function is

$$
J(C, \mu) = \sum_{j=1}^k \sum_{i \in C_j} \|x_i - \mu_j\|^2
$$

where:
- $C_j$ is the $j$-th cluster,
- $\mu_j \in \mathbb{R}^d$ is the center of the cluster.

We aim to minimize $J(C,\mu)$ with respect to $C$ and $\mu$.


## Theorem

If the partition $C_j$ is fixed, then the minimizer of
$$
\sum_{i \in C_j} \|x_i - \mu\|^2
$$

***We, therefore are in a situation where Kmeans is in fact convexe However, it is convex in clusters not overall***

is given by the empirical mean(estimator)

$$
\mu_j^\star = \frac{1}{|C_j|} \sum_{i \in C_j} x_i
$$


## Proof

Fix a cluster $C_j$ and consider the function

$$
\phi(\mu) = \sum_{i \in C_j} \|x_i - \mu\|^2
$$

We expand each squared norm

$$
\|x_i - \mu\|^2
= (x_i - \mu)^T (x_i - \mu)
= x_i^T x_i - 2 x_i^T \mu + \mu^T \mu
$$

Summing over all $i \in C_j$, we obtain

$$
\phi(\mu)
= \sum_{i \in C_j} x_i^T x_i
- 2 \sum_{i \in C_j} x_i^T \mu
+ |C_j| \mu^T \mu
$$

We compute the gradient with respect to $\mu$

$$
\nabla_\mu \phi(\mu)
= -2 \sum_{i \in C_j} x_i + 2 |C_j| \mu
$$

Setting the gradient equal to zero

$$
-2 \sum_{i \in C_j} x_i + 2 |C_j| \mu = 0
$$

This gives

$$
\mu = \frac{1}{|C_j|} \sum_{i \in C_j} x_i
$$

## Conclusion

For each cluster, the center minimizing the intra-cluster sum of squared
distances is the mean of the points assigned to the cluster.

This result explains the update step of the K-means algorithm.


## Non-convexity of the global K-means objective

Let $x_1,\dots,x_n \in \mathbb{R}^d$ be a set of data points and let $k \ge 2$.
The global K-means objective function can be written as

$$
F(\mu_1,\dots,\mu_k)
=
\sum_{i=1}^n \min_{j\in\{1,\dots,k\}} \|x_i-\mu_j\|^2.
$$

This formulation reflects the fact that each data point is assigned to the
closest cluster center.

---

### Definition of convexity

A function $F$ is convex if for all $u,v$ in its domain and for all
$\lambda \in [0,1]$, it satisfies

$$
F(\lambda u + (1-\lambda)v)
\le
\lambda F(u) + (1-\lambda)F(v).
$$

To show that $F$ is not convex, it is sufficient and quite easy to demonstrate it with a counterexample
for which this inequality does not hold.

---

### Counterexample

We consider the simplest non-trivial case:
- dimension $d=1$,
- one data point $n=1$,
- two clusters $k=2$,
- data point $x_1 = 0$.

In this case, the objective function reduces to

$$
F(\mu_1,\mu_2) = \min(\mu_1^2,\mu_2^2).
$$

Let us choose two sets of centers

$$
u = (1,0),
\qquad
v = (0,1).
$$

We compute

$$
F(u) = \min(1,0) = 0,
\qquad
F(v) = \min(0,1) = 0.
$$

Now take $\lambda = \frac{1}{2}$.
The midpoint between $u$ and $v$ is

$$
\frac{u+v}{2} = (0.5,0.5).
$$

Evaluating the objective function at this point yields

$$
F\left(\frac{u+v}{2}\right)
=
\min(0.25,0.25)
=
0.25.
$$

---

### Violation of the convexity inequality

If $F$ were convex, we would have

$$
F\left(\frac{u+v}{2}\right)
\le
\frac{F(u)+F(v)}{2}.
$$

However,

$$
\frac{F(u)+F(v)}{2} = 0,
$$

while

$$
F\left(\frac{u+v}{2}\right) = 0.25 > 0.
$$

This contradicts the convexity inequality.

---

### Conclusion

The convexity inequality is violated for the global K-means objective function.
Therefore, the K-means objective is not convex.


***Starting Kmeans implementation***

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import sys


In [6]:
vers = {
    "numpy" : np.__version__,
    "python" : sys.version_info[:3],
    "matplot" : tuple(map(int,matplotlib.__version__.split(".")[:3]))
    }

In [7]:
def call_vers(d : dict):
    if isinstance(d,dict):
        for name,val in vers.items():
            if name == "python":
                if val > (3,10):
                    print(f"Python sys info correct {val} \n")
            else:
                print(f"Version of {name}:{val}\n")
        return True
    else:
        return False


In [8]:
A = np.random.rand(5,6)
B = np.random.rand(5,6)

X = np.c_[A.ravel(),B.ravel()]
X.ndim


2

In [ ]:
import time
def timer(func):
    def wrapper(*args,**kwargs):
        deb = time.time()
        f = func(*args,**kwargs)
        end = time.time()
        res = end-deb
        return 
    return wrapper

In [28]:
class KMEANS:
    def __init__(self,k,max_iter=10):
        self.k = k
        self.max_iter = max_iter

        self.centroids = None
        self.labels = None
        self.inertia = None

    def fit(self,X):
        X = np.asarray(X)
        sampl = X.shape[0]
        #I select k random values amongst all
        self.centroids = X[np.random.choice(sampl,k,replace=False)]
        for _ in range(self.max_iter):
            self.labels = cluster_assign(X,self.centroids)
            assert self.labels.shape[0] == sampl
            assert np.all((self.labels >= 0) & (self.labels < self.k)) #check if true and 
            NEW_Centroids = centroids_update(X,self.labels,self.k)
            if np.allclose(self.centroids,NEW_Centroids):#to check if the centroids evolved
                break
            self.centroids = NEW_Centroids
        return self
            
    def predict(self,X):

        X = np.asarray(X)
        if self.centroids is None:
            raise ValueError("An error occured, model not fitted")
        return cluster_assign(X,self.centroids)

In [19]:
"""
Fairly simple Euclidianj distance function, using dot product to optimize
"""
def Euclid_Distance(x : np.ndarray, mu : np.ndarray) -> float:
    x = np.asarray(x)
    mu = np.asarray(mu)
    assert x.ndim == 1
    assert mu.ndim == 1
    assert x.shape == mu.shape
    return np.dot(x-mu,x-mu)

In [20]:
x = np.array([0.0, 0.0])
mu = np.array([1.0, 0.0])
Euclid_Distance(x, mu)  # doit donner 1.0

np.float64(1.0)

In [29]:
"""
Simple assignation function, where we check for each variable x(observation) and cs(centroids)
calculate the distance between the observation and each cluster's centroid then return the min
the min shall be used later for the cluster assigning function that shall assign an observation to a cluster
"""

def assignation(x: np.ndarray, centroids: np.ndarray):
    x = np.asarray(x)
    centroids = np.asarray(centroids)

    assert centroids.ndim == 2
    assert x.ndim == 1

    DIST = np.empty(len(centroids))

    for j in range(len(centroids)):
        DIST[j] = Euclid_Distance(x, centroids[j])

    return np.argmin(DIST)

    
    

In [27]:
"""
Quite simple function, assign an observation xi to a centroid by minimizing the distance between obs and centroid see maths up above
"""

def cluster_assign(X, centroids):
    X = np.asarray(X)
    centroids = np.asarray(centroids)

    labels = np.empty(X.shape[0], dtype=int)

    for i in range(X.shape[0]):
        labels[i] = assignation(X[i], centroids)
        
    return labels





In [34]:
"""
For k cluster, I create accumulators + sums and for a label label[i] I retrieve it and add to the appropriate accumulator the value of X
then I calculate the means of my new centroids
"""

def centroids_update(X,labels,k):
    X = np.asarray(X)
    size1,size2 = X.shape
    accum = np.zeros(k,dtype=int)
    sam = np.zeros((k,size2))
   
    for i in range(len(labels)):
        c = labels[i]
        sam[c] += X[i]
        accum[c] += 1

    centroids = np.zeros((k,size2))

    for c in range(k):
        if accum[c] > 0:
            centroids[c] = sam[c]/accum[c]
        else:
            raise ValueError("there are less possible label value than clusters")
    return centroids
    
    
    